In [1]:
import numpy as np 
import pandas as pd 

### read in the full Amazon file (455,000 rows)

In [2]:
amazon = pd.read_csv('Amazon.csv')
print(amazon.shape)

(455000, 13)


### randomly split the dataset into training and test sets

In [3]:
# create training and test sets
from sklearn.model_selection import train_test_split

raw_data_train, raw_data_test, y_train, y_test = train_test_split(amazon, amazon['helpful'], test_size= 0.3, random_state=7)

### check that the file sizes are what you expected

In [4]:
print(raw_data_train.shape) 
print(raw_data_test.shape)
print(y_train.shape)
print(y_test.shape)

(318500, 13)
(136500, 13)
(318500,)
(136500,)


### export as CSV files so you have readily accessible files to read for training and testing

In [5]:
import os
os.system("mkdir -p /home/wolfm2/amazon_data.0/ ") # mw

raw_data_train.to_csv('/home/wolfm2/amazon_data.0/raw_data_train.csv')
raw_data_test.to_csv('/home/wolfm2/amazon_data.0/raw_data_test.csv')
y_train.to_csv('/home/wolfm2/amazon_data.0/y_train.csv')
y_test.to_csv('/home/wolfm2/amazon_data.0/y_test.csv')